In [1]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/parsing/constituency/indon.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/parsing/constituency/texts.txt

In [2]:
import malaya

# electra = malaya.transformer.load(model = 'electra')

In [3]:
vocab_news, embedded_news = malaya.wordvector.load_news()
word_vector_news = malaya.wordvector.load(embedded_news, vocab_news)

In [4]:
with open('texts.txt') as fopen:
    texts = fopen.read().split('\n')

with open('indon.txt') as fopen:
    parsing = fopen.read().split('\n')
    
len(texts), len(parsing)

(1030, 1030)

In [5]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_parsing, test_parsing = train_test_split(texts, parsing, test_size = 0.1)

In [6]:
def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

In [7]:
import re

def remove(string):
    string = string.encode('ascii', errors='ignore').strip().decode()
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string
    
def augment(string, n = 7):
    results = malaya.augmentation.synonym(string)
#     try:
#         results.extend(malaya.augmentation.transformer(string, electra))
#     except:
#         pass
    try:
        results.extend(malaya.augmentation.wordvector(
            string, word_vector_news, soft = True
        ))
    except:
        pass
    results = list(set(results))
    results = [remove(s) for s in results if s.lower() != string.lower()]
    words = string.split()
    results = [s.split() for s in results if len([w for w in s.split() if len(w) > 1]) == len(words)]
    for i in range(len(results)):
        for no, w in enumerate(words):
            results[i][no] = case_of(w)(results[i][no])
        results[i] = ' '.join(results[i])
    return results[:n]

def replace(string, actual_string, aug_string):
    actual_string = actual_string.split()
    aug_string = aug_string.split()
    for no, word in enumerate(actual_string):
        string = string.replace(word, aug_string[no])
    return string

In [8]:
from tqdm import tqdm

results, out = [], []
for i in tqdm(range(len(test_texts))):
    try:
        rows = augment(test_texts[i])
        for row in rows:
            results.append(replace(test_parsing[i], test_texts[i], row))
            out.append(row)
    except:
        pass
    results.append(test_parsing[i])
    out.append(test_texts[i])
    
len(results)

100%|██████████| 103/103 [01:08<00:00,  1.51it/s]


442

In [9]:
with open('test-augmented.txt', 'w') as fopen:
    fopen.write('\n'.join(results))
    
with open('test-augmented-texts.txt', 'w') as fopen:
    fopen.write('\n'.join(out))

In [10]:
from tqdm import tqdm

results, out = [], []
for i in tqdm(range(len(train_texts))):
    try:
        rows = augment(train_texts[i])
        for row in rows:
            results.append(replace(train_parsing[i], train_texts[i], row))
            out.append(row)
    except:
        pass
    results.append(train_parsing[i])
    out.append(train_texts[i])
    
len(results)

100%|██████████| 927/927 [08:35<00:00,  1.80it/s]


3989

In [11]:
with open('train-augmented.txt', 'w') as fopen:
    fopen.write('\n'.join(results))
    
with open('train-augmented-texts.txt', 'w') as fopen:
    fopen.write('\n'.join(out))